In [3]:
import pandas as pd
import numpy as np

from influxdb_client import InfluxDBClient, Point, Dialect

import re
import time
import datetime

import warnings
from influxdb_client.client.warnings import MissingPivotFunction

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.colors as colors

import pandasql as ps
import sqlite3

import csv

import warnings 
from influxdb_client.client.warnings import MissingPivotFunction
warnings.simplefilter("ignore", MissingPivotFunction)

import gc

pd.set_option('display.max_rows', 500)

In [38]:
def _parse_line(line):

    rx_dict = {
    'token': re.compile(r'var token = "(?P<token>.*)"\n'),
    'url': re.compile(r'var url = "(?P<url>.*)"\n'),
    'org': re.compile(r'var org = "(?P<org>.*)"\n'),
    'bucket': re.compile(r'var bucket = "(?P<bucket>.*)"\n'),
    }   

    """
    Do a regex search against all defined regexes and
    return the key and match result of the first matching regex

    """
    for key, rx in rx_dict.items():
        match = rx.search(line)
        if match:
            return key, match
    # if there are no matches
    return None, None

filepath = '/root/flexi-pipe/config.go'
# open the file and read through it line by line
with open(filepath, 'r') as file_object:
    line = file_object.readline()
    while line:
        # at each line check for a match with a regex
        key, match = _parse_line(line)

        if key == 'token':
            token = match.group('token')
        elif key == 'url':
            url = match.group('url')
        elif key == 'org':
            org = match.group('org')
        elif key == 'bucket':
            bucket = match.group('bucket')
        
        line = file_object.readline()
# url="http://192.168.20.58:8086"
url = "http://localhost:8086"

In [39]:
start = 1700580292
end = 1700628416

In [40]:
def experiment(start_time, end_time, filepath):
    # Retrieve experiments data from csv
    data = pd.read_csv(filepath, header=None)
    df = pd.DataFrame(data)

    #Rename columns
    experiments = df.rename(columns={0: "start", 1: "end", 2: "topology", 3: "runtime", 4: "parameter", 5: "d", 6: "dlo", 7: "dhi", 8: "dscore", 9: "dlazy", 10: "dout", 11: "gossipFactor", 12: "initialDelay", 13: "interval"}, errors='raise')

    #Correct timestamp
    experiments["start"] = experiments["start"].str.slice(0, 27)
    experiments["end"] = experiments["end"].str.slice(0, 27)

    #String to timestamp
    # experiments['startUnix'] = pd.to_datetime(experiments["start"],format="%Y-%m-%d %H:%M:%S.%f").astype('int64') / 10**9
    # experiments['endUnix'] = pd.to_datetime(experiments["end"],format="%Y-%m-%d %H:%M:%S.%f").astype('int64') / 10**9
    experiments['startUnix'] = pd.to_datetime(experiments["start"],format="mixed", infer_datetime_format=True).astype('int64') / 10**9
    experiments['endUnix'] = pd.to_datetime(experiments["end"],format="mixed", infer_datetime_format=True).astype('int64') / 10**9


    experiments['startUnix'] = pd.to_timedelta(experiments['startUnix'], unit='s').dt.total_seconds().astype(int)#.astype(str)
    experiments['endUnix'] = pd.to_timedelta(experiments['endUnix'], unit='s').dt.total_seconds().astype(int)#.astype(str)

    #Drop fields we don't mneed for the moment
    exp = experiments.drop(columns=["runtime", "initialDelay"]).sort_values(by=["start"])

    #Get times for different intervals
    # intervals = exp["interval"].drop_duplicates().sort_values().reset_index(drop=True)
    # intervals.head(10)

    expTime = exp[exp['startUnix'].astype(int).between(start_time, end_time)]
    # expTime['experiment'] = expTime.index
    expTime = expTime.reset_index().rename({'index':'experiment'}, axis = 'columns')

    return expTime

experiments = experiment(start, end, '../experiments.csv')
experiments.head(300)

/tmp/ipykernel_6746/3663955728.py:16: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  experiments['startUnix'] = pd.to_datetime(experiments["start"],format="mixed", infer_datetime_format=True).astype('int64') / 10**9
/tmp/ipykernel_6746/3663955728.py:17: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  experiments['endUnix'] = pd.to_datetime(experiments["end"],format="mixed", infer_datetime_format=True).astype('int64') / 10**9


,experiment,start,end,topology,parameter,d,dlo,dhi,dscore,dlazy,dout,gossipFactor,interval,startUnix,endUnix
0,642,2023-11-21 15:24:52.4720765,2023-11-21 15:54:52.4750780,general,informed9,8,6,12,4,6,2,0.15,3.0,1700580292,1700582092
1,643,2023-11-21 15:58:26.2385739,2023-11-21 16:28:26.2470730,general,informed9,8,6,12,4,6,2,0.15,3.0,1700582306,1700584106
2,644,2023-11-21 16:31:59.9970877,2023-11-21 17:02:00.0047743,general,informed9,8,6,12,4,6,2,0.15,3.0,1700584319,1700586120
3,645,2023-11-21 17:05:33.6824106,2023-11-21 17:35:33.6886246,general,informed10,16,6,20,4,6,4,0.50,1.0,1700586333,1700588133
4,646,2023-11-21 17:39:08.0129121,2023-11-21 18:09:08.0166440,general,informed10,16,6,20,4,6,4,0.50,1.0,1700588348,1700590148
5,647,2023-11-21 18:12:42.0079651,2023-11-21 18:42:42.0110142,general,informed10,16,6,20,4,6,4,0.50,1.0,1700590362,1700592162
6,648,2023-11-21 18:46:16.3191492,2023-11-21 19:16:16.3207097,general,informed11,16,8,20,8,8,4,0.15,3.0,1700592376,1700594176
7,649,2023-11-21 19:19:50.6788162,2023-11-21 19:49:50.6887671,general,informed11,16,8,20,8,8,4,0.15,3.0,1700594390,1700596190
8,650,2023-11-21 19:53:25.1009466,2023-11-21 20:23:25.1072124,general,informed11,16,8,20,8,8,4,0.15,3.0,1700596405,1700598205
9,651,2023-11-21 20:26:59.6970253,2023-11-21 20:56:59.7013529,general,informed12,16,8,15,16,8,4,0.75,0.5,1700598419,1700600219


In [41]:
def from_influx(url, token, org, measurement, start_time, end_time,grouping_key):
    client = InfluxDBClient(url=url, token=token, org=org,  timeout=900_000)

    # write_api = client.write_api(write_options=SYNCHRONOUS)
    query_api = client.query_api()

    data_frame = query_api.query_data_frame('from(bucket: "gs") '
                                        ' |> range(start: '+str(start_time)+', stop:'+str(end_time)+') '
                                        ' |> filter(fn: (r) => r._measurement == "'+measurement+'") '
                                        ' |> group(columns: ["_measurement", "_field"], mode: "by") '
                                        ' |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")')
    client.close()

    # df = data_frame.drop(columns=['result', 'table','_start', '_stop', '_measurement', 'topic', 'receivedFrom']).sort_values(by=["_time"]).reset_index(drop=True)
    data_frame.reset_index(inplace=True)
    df = data_frame[['_time', grouping_key]].sort_values(by=["_time"]).reset_index(drop=True)
    df["_time"] = pd.to_datetime(df["_time"])

    return df

In [42]:
def from_influx_count(url, token, org, start_time, end_time,grouping_key):
    # start_time = 1693222601 
    # end_time = 1693228430    
    client = InfluxDBClient(url=url, token=token, org=org,  timeout=900_000)
        
        # write_api = client.write_api(write_options=SYNCHRONOUS)
    query_api = client.query_api()
        
    data_frame = query_api.query_data_frame('from(bucket: "gs") '
                       ' |> range(start: '+str(start_time)+', stop:'+str(end_time)+') '
                        '|> filter(fn: (r) => r._measurement == "deliverMessage") '
                        '|> group(columns: ["_measurement", "_field"], mode: "by") '
                        '|> count()')
    client.close()

    if data_frame.empty:
        count = 0
    else:
        count = data_frame["_value"].min().astype(int)
    return count

In [43]:
#Validate data
validate = pd.DataFrame()

for index, row in experiments.iterrows():
    count = from_influx_count(url, token, org, row["startUnix"], row["endUnix"],"_measurement")
    d = {'experiment': [row['experiment']], 'count': [count], 'topology': [row["topology"]], 'd': [row["d"]],'dhi': [row["dhi"]],'dlo': [row["dlo"]],'dlazy': [row["dlazy"]],
        'dscore': [row["dscore"]],'dout': [row["dout"]],'gossipFactor': [row["gossipFactor"]],'interval': [row["interval"]],}
    aux = pd.DataFrame(data=d)
    validate = pd.concat([validate, aux])

# validate.head(200)

exps = experiments.merge(validate, on=['experiment', 'topology','d','dhi','dlo','dlazy','dscore','dout','gossipFactor','interval'])
exps = exps.loc[exps["count"]>0]

exps.head(10)

,experiment,start,end,topology,parameter,d,dlo,dhi,dscore,dlazy,dout,gossipFactor,interval,startUnix,endUnix,count
0,642,2023-11-21 15:24:52.4720765,2023-11-21 15:54:52.4750780,general,informed9,8,6,12,4,6,2,0.15,3.0,1700580292,1700582092,115113
1,643,2023-11-21 15:58:26.2385739,2023-11-21 16:28:26.2470730,general,informed9,8,6,12,4,6,2,0.15,3.0,1700582306,1700584106,94376
2,644,2023-11-21 16:31:59.9970877,2023-11-21 17:02:00.0047743,general,informed9,8,6,12,4,6,2,0.15,3.0,1700584319,1700586120,90696
3,645,2023-11-21 17:05:33.6824106,2023-11-21 17:35:33.6886246,general,informed10,16,6,20,4,6,4,0.50,1.0,1700586333,1700588133,95091
4,646,2023-11-21 17:39:08.0129121,2023-11-21 18:09:08.0166440,general,informed10,16,6,20,4,6,4,0.50,1.0,1700588348,1700590148,124132
5,647,2023-11-21 18:12:42.0079651,2023-11-21 18:42:42.0110142,general,informed10,16,6,20,4,6,4,0.50,1.0,1700590362,1700592162,110577
6,648,2023-11-21 18:46:16.3191492,2023-11-21 19:16:16.3207097,general,informed11,16,8,20,8,8,4,0.15,3.0,1700592376,1700594176,149392
10,652,2023-11-21 21:00:32.4770009,2023-11-21 21:30:32.4890113,general,informed12,16,8,15,16,8,4,0.75,0.5,1700600432,1700602232,17167
11,653,2023-11-21 21:34:06.4672540,2023-11-21 22:04:06.4816988,general,informed12,16,8,15,16,8,4,0.75,0.5,1700602446,1700604246,86607
12,654,2023-11-21 22:07:40.2346165,2023-11-21 22:37:40.2385545,general,informed13,5,4,7,3,2,2,0.25,1.0,1700604460,1700606260,84236


In [44]:
#Remove fault executions

#get median
median = exps.drop(columns=['start','end','startUnix','endUnix', 'parameter', 'experiment']).drop_duplicates()
median = median.groupby(['topology','d','dhi','dlo','dlazy','dscore','dout','gossipFactor','interval']).agg({'count':['median', 'std', 'mean']})
median.columns = median.columns.droplevel(0)
median.reset_index(inplace=True)
median = median.loc[median['median'] > 0]

median.head(50)

,topology,d,dhi,dlo,dlazy,dscore,dout,gossipFactor,interval,median,std,mean
0,general,5,7,4,2,3,2,0.25,1.0,84236.0,33491.111970,95775.000000
1,general,6,8,3,2,2,4,0.25,0.5,37796.5,42383.273358,37796.500000
2,general,6,8,4,5,7,4,0.25,3.0,38317.0,45556.061485,38317.000000
3,general,8,12,6,6,4,2,0.15,3.0,94376.0,13164.063823,100061.666667
4,general,16,15,8,8,16,4,0.75,0.5,51887.0,49101.494886,51887.000000
5,general,16,20,6,6,4,4,0.50,1.0,110577.0,14531.195764,109933.333333
6,general,16,20,8,8,8,4,0.15,3.0,149392.0,NaN,149392.000000
7,general,21,24,16,16,5,8,0.50,1.0,46730.0,50286.605851,46730.000000


In [45]:
#Validate the data

validation = exps.merge(median, on=['topology','d','dhi','dlo','dlazy','dscore','dout','gossipFactor','interval'])
validated = validation[(validation['count'] >= validation['mean']-(0.15*validation['std']))]
validated = validated.drop(columns=['startUnix', 'endUnix'])
validated.dropna()

validated.head(100)

,experiment,start,end,topology,parameter,d,dlo,dhi,dscore,dlazy,dout,gossipFactor,interval,count,median,std,mean
0,642,2023-11-21 15:24:52.4720765,2023-11-21 15:54:52.4750780,general,informed9,8,6,12,4,6,2,0.15,3.0,115113,94376.0,13164.063823,100061.666667
4,646,2023-11-21 17:39:08.0129121,2023-11-21 18:09:08.0166440,general,informed10,16,6,20,4,6,4,0.50,1.0,124132,110577.0,14531.195764,109933.333333
5,647,2023-11-21 18:12:42.0079651,2023-11-21 18:42:42.0110142,general,informed10,16,6,20,4,6,4,0.50,1.0,110577,110577.0,14531.195764,109933.333333
8,653,2023-11-21 21:34:06.4672540,2023-11-21 22:04:06.4816988,general,informed12,16,8,15,16,8,4,0.75,0.5,86607,51887.0,49101.494886,51887.000000
11,656,2023-11-21 23:14:48.2294649,2023-11-21 23:44:48.2363093,general,informed13,5,4,7,3,2,2,0.25,1.0,133510,84236.0,33491.111970,95775.000000
12,657,2023-11-21 23:48:22.6068989,2023-11-22 00:18:22.6099262,general,informed14,6,4,8,7,5,4,0.25,3.0,70530,38317.0,45556.061485,38317.000000
15,662,2023-11-22 02:36:14.2823737,2023-11-22 03:06:14.2932442,general,informed15,6,3,8,2,2,4,0.25,0.5,67766,37796.5,42383.273358,37796.500000
16,663,2023-11-22 03:09:48.5789046,2023-11-22 03:39:48.5860069,general,informed16,21,16,24,5,16,8,0.50,1.0,82288,46730.0,50286.605851,46730.000000


In [46]:
final = validated.drop(columns=['experiment', 'start','end', 'parameter', 'count', 'std', 'median', 'mean']).drop_duplicates()
final['n_nodes'] = 23
final = final.reset_index().rename({'index':'identifier'}, axis = 'columns')
final.head(100)

,identifier,topology,d,dlo,dhi,dscore,dlazy,dout,gossipFactor,interval,n_nodes
0,0,general,8,6,12,4,6,2,0.15,3.0,23
1,4,general,16,6,20,4,6,4,0.50,1.0,23
2,8,general,16,8,15,16,8,4,0.75,0.5,23
3,11,general,5,4,7,3,2,2,0.25,1.0,23
4,12,general,6,4,8,7,5,4,0.25,3.0,23
5,15,general,6,3,8,2,2,4,0.25,0.5,23
6,16,general,21,16,24,5,16,8,0.50,1.0,23


In [47]:
#Limit extraction to 15 minutes

experiments = validated.drop(columns=['count','median', 'mean','std']).rename(columns={'start':'originalStart','end':'originalEnd'})

experiments['originalStart'] = pd.to_datetime(experiments["originalStart"], format='mixed')
experiments['originalEnd'] = pd.to_datetime(experiments["originalEnd"], format='mixed')

experiments['start'] = experiments['originalStart'] + pd.Timedelta(hours=0, minutes=7, seconds=30)
experiments['end'] = experiments['originalEnd'] - pd.Timedelta(hours=0, minutes=7, seconds=30)

experiments['startUnix'] = pd.to_datetime(experiments["start"],format="mixed").astype('int64') / 10**9
experiments['endUnix'] = pd.to_datetime(experiments["end"],format="mixed").astype('int64') / 10**9

experiments['startUnix'] = pd.to_timedelta(experiments['startUnix'], unit='s').dt.total_seconds().astype(int)#.astype(str)
experiments['endUnix'] = pd.to_timedelta(experiments['endUnix'], unit='s').dt.total_seconds().astype(int)#.astype(str)

# experiments.to_csv('experiments_filtered.csv')
experiments.head(10)

,experiment,originalStart,originalEnd,topology,parameter,d,dlo,dhi,dscore,dlazy,dout,gossipFactor,interval,start,end,startUnix,endUnix
0,642,2023-11-21 15:24:52.472076500,2023-11-21 15:54:52.475078000,general,informed9,8,6,12,4,6,2,0.15,3.0,2023-11-21 15:32:22.472076500,2023-11-21 15:47:22.475078000,1700580742,1700581642
4,646,2023-11-21 17:39:08.012912100,2023-11-21 18:09:08.016644000,general,informed10,16,6,20,4,6,4,0.50,1.0,2023-11-21 17:46:38.012912100,2023-11-21 18:01:38.016644000,1700588798,1700589698
5,647,2023-11-21 18:12:42.007965100,2023-11-21 18:42:42.011014200,general,informed10,16,6,20,4,6,4,0.50,1.0,2023-11-21 18:20:12.007965100,2023-11-21 18:35:12.011014200,1700590812,1700591712
8,653,2023-11-21 21:34:06.467254000,2023-11-21 22:04:06.481698800,general,informed12,16,8,15,16,8,4,0.75,0.5,2023-11-21 21:41:36.467254000,2023-11-21 21:56:36.481698800,1700602896,1700603796
11,656,2023-11-21 23:14:48.229464900,2023-11-21 23:44:48.236309300,general,informed13,5,4,7,3,2,2,0.25,1.0,2023-11-21 23:22:18.229464900,2023-11-21 23:37:18.236309300,1700608938,1700609838
12,657,2023-11-21 23:48:22.606898900,2023-11-22 00:18:22.609926200,general,informed14,6,4,8,7,5,4,0.25,3.0,2023-11-21 23:55:52.606898900,2023-11-22 00:10:52.609926200,1700610952,1700611852
15,662,2023-11-22 02:36:14.282373700,2023-11-22 03:06:14.293244200,general,informed15,6,3,8,2,2,4,0.25,0.5,2023-11-22 02:43:44.282373700,2023-11-22 02:58:44.293244200,1700621024,1700621924
16,663,2023-11-22 03:09:48.578904600,2023-11-22 03:39:48.586006900,general,informed16,21,16,24,5,16,8,0.50,1.0,2023-11-22 03:17:18.578904600,2023-11-22 03:32:18.586006900,1700623038,1700623938


In [48]:
def from_influx_message(url, token, org, start_time, end_time):
    client = InfluxDBClient(url=url, token=token, org=org,  timeout=900_000)

    # write_api = client.write_api(write_options=SYNCHRONOUS)
    query_api = client.query_api()

    data_frame = query_api.query_data_frame('from(bucket: "gs") '
                                        ' |> range(start: '+str(start_time)+', stop:'+str(end_time)+') '
                                        ' |> filter(fn: (r) => r._measurement == "message" and r._field == "type" and (r._value == 11 or r._value == 12 or r._value == 3 or r._value == 2)) '
                                        ' |> group(columns: ["_measurement", "_field"], mode: "by") '
                                        ' |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")')
    client.close()

    # df = data_frame.drop(columns=['result', 'table','_start', '_stop', '_measurement', 'topic', 'receivedFrom']).sort_values(by=["_time"]).reset_index(drop=True)
    data_frame.reset_index(inplace=True)
    df = data_frame[['_time', 'type']].sort_values(by=["_time"]).reset_index(drop=True)
    df["_time"] = pd.to_datetime(df["_time"])

    return df

In [49]:
message = pd.DataFrame()
execs = experiments.drop(columns=['originalStart', 'originalEnd'])
# final=final.loc[final["identifier"] == 1]

for index, row in final.iterrows():
    # print(row["start"], row["end"], row["experiment"])
    execs = exps.loc[(exps["topology"] == row["topology"]) & (exps["d"] == row["d"]) &
                            (exps["dlo"] == row["dlo"]) & (exps["dhi"] == row["dhi"]) &
                            (exps["dscore"] == row["dscore"]) & (exps["dlazy"] == row["dlazy"]) &
                            (exps["dout"] == row["dout"]) & (exps["gossipFactor"] == row["gossipFactor"]) &                    
                            (exps["interval"] == row["interval"])]

    for idx, exec in execs.iterrows():
        # print(exec["startUnix"], exec["endUnix"])
        query_mess = from_influx_message(url, token, org, exec["startUnix"], exec["endUnix"])

        query_mess["identifier"] = row["identifier"]

        query_mess["experiment"] = exec["experiment"]

        query_mess["start"] = exec["start"]
        query_mess["end"] = exec["end"]
            
        query_mess["_time"] = pd.to_datetime(query_mess["_time"])
        message = pd.concat([message,query_mess])

        query_mess.to_csv('./mess_'+str(exec["experiment"])+'.csv')
        
        del query_mess
        # del deliverMessage
        gc.collect()

message.head(10)

,_time,type,identifier,experiment,start,end
0,2023-11-21 15:24:52.415456+00:00,3.0,0,642,2023-11-21 15:24:52.4720765,2023-11-21 15:54:52.4750780
1,2023-11-21 15:24:52.416362+00:00,3.0,0,642,2023-11-21 15:24:52.4720765,2023-11-21 15:54:52.4750780
2,2023-11-21 15:24:52.417290+00:00,2.0,0,642,2023-11-21 15:24:52.4720765,2023-11-21 15:54:52.4750780
3,2023-11-21 15:24:52.417292+00:00,3.0,0,642,2023-11-21 15:24:52.4720765,2023-11-21 15:54:52.4750780
4,2023-11-21 15:24:52.417518+00:00,2.0,0,642,2023-11-21 15:24:52.4720765,2023-11-21 15:54:52.4750780
5,2023-11-21 15:24:52.417613+00:00,2.0,0,642,2023-11-21 15:24:52.4720765,2023-11-21 15:54:52.4750780
6,2023-11-21 15:24:52.417636+00:00,3.0,0,642,2023-11-21 15:24:52.4720765,2023-11-21 15:54:52.4750780
7,2023-11-21 15:24:52.417647+00:00,2.0,0,642,2023-11-21 15:24:52.4720765,2023-11-21 15:54:52.4750780
8,2023-11-21 15:24:52.417676+00:00,2.0,0,642,2023-11-21 15:24:52.4720765,2023-11-21 15:54:52.4750780
9,2023-11-21 15:24:52.417758+00:00,2.0,0,642,2023-11-21 15:24:52.4720765,2023-11-21 15:54:52.4750780


In [50]:
message.to_csv('./messageTimeSeries.csv', mode='a')

In [51]:
del message
gc.collect()

0

In [52]:
def from_influx(url, token, org, measurement, start_time, end_time,grouping_key):
    client = InfluxDBClient(url=url, token=token, org=org,  timeout=900_000)

    # write_api = client.write_api(write_options=SYNCHRONOUS)
    query_api = client.query_api()

    data_frame = query_api.query_data_frame('from(bucket: "gs") '
                                        ' |> range(start: '+str(start_time)+', stop:'+str(end_time)+') '
                                        ' |> filter(fn: (r) => r._measurement == "'+measurement+'") '
                                        ' |> group(columns: ["_measurement", "_field"], mode: "by") '
                                        ' |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")')
    client.close()

    # df = data_frame.drop(columns=['result', 'table','_start', '_stop', '_measurement', 'topic', 'receivedFrom']).sort_values(by=["_time"]).reset_index(drop=True)
    if data_frame.empty == False:
        data_frame.reset_index(inplace=True)
        df = data_frame[['_time', grouping_key]].sort_values(by=["_time"]).reset_index(drop=True)
        df["_time"] = pd.to_datetime(df["_time"])
    else:
        df = data_frame

    return df

In [59]:
gossip = pd.DataFrame()
execs = experiments.drop(columns=['originalStart', 'originalEnd'])

for index, row in final.iterrows():
    # print(row["start"], row["end"], row["experiment"])
    execs = exps.loc[(exps["topology"] == row["topology"]) & (exps["d"] == row["d"]) &
                            (exps["dlo"] == row["dlo"]) & (exps["dhi"] == row["dhi"]) &
                            (exps["dscore"] == row["dscore"]) & (exps["dlazy"] == row["dlazy"]) &
                            (exps["dout"] == row["dout"]) & (exps["gossipFactor"] == row["gossipFactor"]) &                    
                            (exps["interval"] == row["interval"])]

    for idx, exec in execs.iterrows():
        # print(exec["startUnix"], exec["endUnix"])
        query_iwant = from_influx(url, token, org, "iwant", exec["startUnix"], exec["endUnix"],"_measurement")

        if query_iwant.empty == False:
            query_iwant["identifier"] = row["identifier"]
            query_iwant["experiment"] = exec["experiment"]
 
            query_iwant["_time"] = pd.to_datetime(query_iwant["_time"])
            gossip = pd.concat([gossip, query_iwant])

            query_iwant.to_csv('./iwant_'+str(exec["experiment"])+'.csv')z

            del query_iwant

        query_ihave = from_influx(url, token, org, "ihave", exec["startUnix"], exec["endUnix"],"_measurement")
        if query_ihave.empty == False:
            query_ihave["identifier"] = row["identifier"]
            query_ihave["experiment"] = exec["experiment"]

            query_ihave["_time"] = pd.to_datetime(query_ihave["_time"])
            gossip = pd.concat([gossip, query_ihave])

            query_ihave.to_csv('./ihave_'+str(exec["experiment"])+'.csv')
        
            del query_ihave

        gc.collect()

gossip.head(10)

,_time,_measurement,identifier,experiment
0,2023-11-21 15:24:53.032623+00:00,iwant,0,642
1,2023-11-21 15:40:26.891377+00:00,iwant,0,642
0,2023-11-21 15:24:52.988402+00:00,ihave,0,642
1,2023-11-21 15:24:52.989674+00:00,ihave,0,642
2,2023-11-21 15:24:52.990178+00:00,ihave,0,642
3,2023-11-21 15:24:52.990617+00:00,ihave,0,642
4,2023-11-21 15:24:52.992429+00:00,ihave,0,642
5,2023-11-21 15:24:52.992519+00:00,ihave,0,642
6,2023-11-21 15:24:52.993070+00:00,ihave,0,642
7,2023-11-21 15:24:52.993105+00:00,ihave,0,642


In [60]:
gossip.to_csv('./gossipTimeSeries.csv', mode='a')